In [1]:
import numpy as np
import matplotlib.pyplot as plt

from tensorflow import data

from keras.utils import image_dataset_from_directory, load_img, img_to_array
from keras.ops import expand_dims
from keras.applications import ResNet50

In [ ]:
image_size = (227, 227)
batch_size = 128

train_ds, val_ds = image_dataset_from_directory(
    "dataset",
    validation_split=0.2,
    subset="both",
    seed=1337,
    image_size=image_size,
    batch_size=batch_size,
    label_mode="categorical",
)

classes = train_ds.class_names
input_shape = image_size + (3,)

In [3]:
def one_hot_decode(one_hot):
    return np.argmax(one_hot, axis=-1)

In [ ]:
plt.figure(figsize=(10, 10))
for images, labels in train_ds.take(1):
    for i in range(9):
        image = images[i].numpy().astype("uint8")
        label = one_hot_decode(labels[i])

        ax = plt.subplot(3, 3, i + 1)
        plt.imshow(image)
        plt.title(f"{classes[label]} ({label})")
        plt.axis("off")

In [5]:
# Prefetching samples in GPU memory helps maximize GPU utilization.
train_ds = train_ds.prefetch(data.AUTOTUNE)
val_ds = val_ds.prefetch(data.AUTOTUNE)

In [6]:
model = ResNet50(
    input_shape=input_shape,
    weights=None,
    classes=len(classes),
)

In [ ]:
model.summary()

In [8]:
model.compile(
    optimizer="adam",
    loss="categorical_crossentropy",
    metrics=["accuracy"],
)

In [ ]:
model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=10,
)

In [ ]:
img = load_img("PetImages/Cat/10198.jpg", target_size=image_size)
plt.imshow(img)

img_array = img_to_array(img)
img_array = expand_dims(img_array, 0)  # Create batch axis

predictions = model.predict(img_array)
print(predictions)